In [1]:
# Transfer Learning
# Add a new layer (or a few new layers) on a pretrain model, and only train the last layer.
# Or modify (re-train) the last few layers of the pre-train model.

# e.g. AlexNet: CNN-18 only re-train the last three fully connected layers

In [2]:
# Please download the hymenoptera dataset first: https://www.kaggle.com/datasets/thedatasith/hymenoptera
# ImageFolder
# Scheduler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
  'train': transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
  ]),
  'val': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
  ]),
}

# import data
# in windows
data_dir = 'C:\\Users\\bansh\\Sites\\Active_Project\\practice-pytorch\\src\\data\\hymenoptera_data\\hymenoptera_data'
sets = ['train', 'val']
image_datasets = {
  x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']
}
dataloaders = {
  x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'val']
}
dataset_sizes = {
  x: len(image_datasets[x]) for x in ['train', 'val']
}
class_names = image_datasets['train'].classes
print(class_names)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs-1}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train() # set model to training mode
      else:
        model.eval()  # set model to evaluate mode

      running_loss = 0.0
      running_corrects = 0

      # Iterate over data.
      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          # backward + optimize only if in training phase
          if phase == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
      
      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

      # deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

    print()

  time_elapsed = time.time() - since
  print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
  print(f'Best val Acc: {best_acc:4f}')

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

['ants', 'bees']


In [5]:
# 1. Fine tuning: train the entire model again but only for a little bit

# Original Model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

# Transfer Learning
model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# scheduler 
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # every 7 epochs, learning rate reduces by 0.1

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.5922 Acc: 0.6926
val Loss: 0.4307 Acc: 0.8235

Epoch 1/19
----------
train Loss: 0.4982 Acc: 0.7541
val Loss: 0.3386 Acc: 0.9150

Epoch 2/19
----------
train Loss: 0.4817 Acc: 0.7746
val Loss: 0.2896 Acc: 0.9346

Epoch 3/19
----------
train Loss: 0.4468 Acc: 0.8033
val Loss: 0.2341 Acc: 0.9412

Epoch 4/19
----------
train Loss: 0.3398 Acc: 0.8607
val Loss: 0.2240 Acc: 0.9346

Epoch 5/19
----------
train Loss: 0.4427 Acc: 0.8115
val Loss: 0.2126 Acc: 0.9412

Epoch 6/19
----------
train Loss: 0.3776 Acc: 0.8566
val Loss: 0.2028 Acc: 0.9412

Epoch 7/19
----------
train Loss: 0.3572 Acc: 0.8648
val Loss: 0.2050 Acc: 0.9412

Epoch 8/19
----------
train Loss: 0.3328 Acc: 0.8648
val Loss: 0.1962 Acc: 0.9412

Epoch 9/19
----------
train Loss: 0.3376 Acc: 0.8525
val Loss: 0.1900 Acc: 0.9412

Epoch 10/19
----------
train Loss: 0.3638 Acc: 0.8238
val Loss: 0.1901 Acc: 0.9412

Epoch 11/19
----------
train Loss: 0.3459 Acc: 0.8566
val Loss: 0.2070 Acc: 0.9412

Ep

In [4]:
# 2. Freeze all the upper layers and only train the last few layers (faster)

# Original Model
model = models.resnet18(pretrained=True)

# Freeze layers
for param in model.parameters():
  param.requires_grad = False

num_ftrs = model.fc.in_features

# Transfer Learning
model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# scheduler 
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # every 7 epochs, learning rate reduces by 0.1

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.7704 Acc: 0.4877
val Loss: 0.6448 Acc: 0.6275

Epoch 1/19
----------
train Loss: 0.6337 Acc: 0.6311
val Loss: 0.4654 Acc: 0.8039

Epoch 2/19
----------
train Loss: 0.5401 Acc: 0.7787
val Loss: 0.4020 Acc: 0.8758

Epoch 3/19
----------
train Loss: 0.5194 Acc: 0.7459
val Loss: 0.3453 Acc: 0.9085

Epoch 4/19
----------
train Loss: 0.4759 Acc: 0.7705
val Loss: 0.3122 Acc: 0.9150

Epoch 5/19
----------
train Loss: 0.4653 Acc: 0.7951
val Loss: 0.2686 Acc: 0.9477

Epoch 6/19
----------
train Loss: 0.4250 Acc: 0.8156
val Loss: 0.2590 Acc: 0.9412

Epoch 7/19
----------
train Loss: 0.3958 Acc: 0.8320
val Loss: 0.2740 Acc: 0.9346

Epoch 8/19
----------
train Loss: 0.4427 Acc: 0.8238
val Loss: 0.2506 Acc: 0.9412

Epoch 9/19
----------
train Loss: 0.4306 Acc: 0.8074
val Loss: 0.2616 Acc: 0.9346

Epoch 10/19
----------
train Loss: 0.3780 Acc: 0.8648
val Loss: 0.2604 Acc: 0.9477

Epoch 11/19
----------
train Loss: 0.4462 Acc: 0.7992
val Loss: 0.2705 Acc: 0.9085

Ep